<a href="https://colab.research.google.com/github/amzesmoro/content_base/blob/master/Proyek_PBA_YouTube_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Proyek PBA. Kelompok 2 - Youtube Recommender System**

## **Load Data**


In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

**Check Files on Drive**

In [2]:
file_list = drive.ListFile({'q': "'1dMlcirgEfFW6_0tGe21MyT4bwHuoL3Ba' in parents and trashed=false"}).GetList()
for data_file in file_list:
  print('title: %s, id: %s' % (data_file['title'], data_file['id']))

title: us_category.csv, id: 1CXOXm7gkOt3aWdih1nbITFZSz7nKyxW5
title: gb_category.csv, id: 1eHeKD1eVIfZyUL175u0jhXC-mSOa3e7x
title: US_category_id.json, id: 1Tc77q9L_7FQFMY5FW2nhdEGcfAxd3dCo
title: GB_category_id.json, id: 1OXbhBVdDLjBmsL2hov-5Sl5Ljrks4mv7
title: USvideos.csv, id: 1DPxy5cEjvxeVqVQXOVEEWAR60PwOwzAo
title: GBvideos.csv, id: 1EORpXxrwN63P_9Mo3nDl8W-WrMp1gATQ


**Read Data From GDrive**

In [0]:
data_us = drive.CreateFile({'id': '1DPxy5cEjvxeVqVQXOVEEWAR60PwOwzAo'})
data_us.GetContentFile('USvideos.csv')
data_us_category_json = drive.CreateFile({'id': '1Tc77q9L_7FQFMY5FW2nhdEGcfAxd3dCo'})
data_us_category_json.GetContentFile('US_category_id.json')
data_gb = drive.CreateFile({'id': '1EORpXxrwN63P_9Mo3nDl8W-WrMp1gATQ'})
data_gb.GetContentFile('GBvideos.csv')
data_gb_category_json = drive.CreateFile({'id': '1OXbhBVdDLjBmsL2hov-5Sl5Ljrks4mv7'})
data_gb_category_json.GetContentFile('GB_category_id.json')
data_us_category = drive.CreateFile({'id': '1CXOXm7gkOt3aWdih1nbITFZSz7nKyxW5'})
data_us_category.GetContentFile('us_category.csv')
data_gb_category = drive.CreateFile({'id': '1eHeKD1eVIfZyUL175u0jhXC-mSOa3e7x'})
data_gb_category.GetContentFile('gb_category.csv')

In [0]:
import numpy as np
import json  
import pandas as pd
from pandas.io.json import json_normalize

**Read Dataset USVideos**

In [5]:
df_us = pd.read_csv('USvideos.csv')
df_us['country'] = 'USA'
df_us.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,USA
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",USA
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,USA
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,USA
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,USA


**Read JSON from Category of US Dataset**

In [6]:
with open('US_category_id.json') as df_us_category_id:
    us_category = json.load(df_us_category_id)
dataframe_us_category = json_normalize(us_category['items'])
select_feature_us_category = ['id', 'snippet.title']
df_us_category = dataframe_us_category[select_feature_us_category]
df_us_category = df_us_category.rename(columns={"id":"category_id", "snippet.title":"category_title"})
df_us_category.head()

,category_id,category_title
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


**Read Dataset GBVideos**

In [7]:
df_gb = pd.read_csv('GBvideos.csv')
df_gb['country'] = 'Great Britain'
df_gb.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,Jw1Y-zhQURU,17.14.11,John Lewis Christmas Ad 2017 - #MozTheMonster,John Lewis,26,2017-11-10T07:38:29.000Z,"christmas|""john lewis christmas""|""john lewis""|...",7224515,55681,10247,9479,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,False,False,False,Click here to continue the story and make your...,Great Britain
1,3s1rvMFUweQ,17.14.11,Taylor Swift: …Ready for It? (Live) - SNL,Saturday Night Live,24,2017-11-12T06:24:44.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",1053632,25561,2294,2757,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,False,False,False,Musical guest Taylor Swift performs …Ready for...,Great Britain
2,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,Great Britain
3,PUTEiSjKwJU,17.14.11,Goals from Salford City vs Class of 92 and Fri...,Salford City Football Club,17,2017-11-13T02:30:38.000Z,"Salford City FC|""Salford City""|""Salford""|""Clas...",27833,193,12,37,https://i.ytimg.com/vi/PUTEiSjKwJU/default.jpg,False,False,False,Salford drew 4-4 against the Class of 92 and F...,Great Britain
4,rHwDegptbI4,17.14.11,Dashcam captures truck's near miss with child ...,Cute Girl Videos,25,2017-11-13T01:45:13.000Z,[none],9815,30,2,30,https://i.ytimg.com/vi/rHwDegptbI4/default.jpg,False,False,False,Dashcam captures truck's near miss with child ...,Great Britain


**Read JSON from Category of GB Dataset**

In [8]:
with open('GB_category_id.json') as df_gb_category_id:
    gb_category = json.load(df_gb_category_id)
dataframe_gb_category = json_normalize(gb_category['items'])
select_feature_gb_category = ['id', 'snippet.title']
df_gb_category = dataframe_gb_category[select_feature_gb_category]
df_gb_category = df_gb_category.rename(columns={"id":"category_id", "snippet.title":"category_title"})
df_gb_category.head()

,category_id,category_title
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


**Read CSV Category US dan GB**

In [0]:
df_us_category = pd.read_csv('us_category.csv')
df_gb_category = pd.read_csv('gb_category.csv')

## **US Videos**

In [0]:
# df_us_category['category_id'] = df_us_category['category_id'].apply(int)
df_us = pd.merge(df_us, df_us_category, on=['category_id'])
sort_columns = ['video_id', 'channel_title', 'title', 'category_id', 'category_title', 'tags', 'description', 'views', 'likes', 'dislikes', 'thumbnail_link', 'country']

In [11]:
df_us = df_us[sort_columns]
df_us.head()

,video_id,channel_title,title,category_id,category_title,tags,description,views,likes,dislikes,thumbnail_link,country
0,2kyS6SvSYSE,CaseyNeistat,WE WANT TO TALK ABOUT OUR MARRIAGE,22,People & Blogs,SHANtell martin,SHANTELL'S CHANNEL - https://www.youtube.com/s...,748374,57527,2966,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,USA
1,0mlNzVSJrT0,Nobrand,Me-O Cats Commercial,22,People & Blogs,"cute|""cats""|""thai""|""eggs""",Kittens come out of the eggs in a Thai commerc...,98966,2486,184,https://i.ytimg.com/vi/0mlNzVSJrT0/default.jpg,USA
2,STI2fI7sKMo,Shawn Johnson East,"AFFAIRS, EX BOYFRIENDS, $18MILLION NET WORTH -...",22,People & Blogs,"shawn johnson|""andrew east""|""shawn east""|""shaw...",Subscribe for weekly videos ▶ http://bit.ly/sj...,321053,4451,1772,https://i.ytimg.com/vi/STI2fI7sKMo/default.jpg,USA
3,KODzih-pYlU,Grace Helbig,BLIND(folded) CAKE DECORATING CONTEST (with Mo...,22,People & Blogs,"itsgrace|""funny""|""comedy""|""vlog""|""grace""|""helb...",Molly is an god damn amazing human and she cha...,197062,7250,217,https://i.ytimg.com/vi/KODzih-pYlU/default.jpg,USA
4,8mhTWqWlQzU,Safiya Nygaard,Wearing Online Dollar Store Makeup For A Week,22,People & Blogs,wearing online dollar store makeup for a week|...,I found this online dollar store called ShopMi...,2744430,115426,1110,https://i.ytimg.com/vi/8mhTWqWlQzU/default.jpg,USA


## **GB Videos**

In [0]:
# df_gb_category['category_id'] = df_gb_category['category_id'].apply(int)
df_gb = pd.merge(df_gb, df_gb_category, on=['category_id'])
sort_columns = ['video_id', 'channel_title', 'title', 'category_id', 'category_title', 'tags', 'description', 'views', 'likes', 'dislikes', 'thumbnail_link', 'country']

In [13]:
df_gb = df_gb[sort_columns]
df_gb.head()

,video_id,channel_title,title,category_id,category_title,tags,description,views,likes,dislikes,thumbnail_link,country
0,Jw1Y-zhQURU,John Lewis,John Lewis Christmas Ad 2017 - #MozTheMonster,26,Howto & Style,"christmas|""john lewis christmas""|""john lewis""|...",Click here to continue the story and make your...,7224515,55681,10247,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,Great Britain
1,MyEqfBkzESU,John Maclean,EXTREME GOLDEN TAN TRANSFORMATION,26,Howto & Style,"Extreme Tan|""Tanning""|""Tanning Bed""|""How to ge...",-\n\n\nWebsite - http://www.johnmaclean.co.uk\...,300617,8133,2693,https://i.ytimg.com/vi/MyEqfBkzESU/default.jpg,Great Britain
2,b4vTZx_AtHk,HowToBasic,How To Dry a Shirt in 30 Seconds,26,Howto & Style,"how to dry a shirt in 30 seconds|""how to dry a...",Today I show you how to completely dry a shirt...,2063667,98804,11682,https://i.ytimg.com/vi/b4vTZx_AtHk/default.jpg,Great Britain
3,OWvCCu2t6Ts,Sprinkleofglitter,SO Much To Tell You | 32 Weeks Pregnant | MOTH...,26,Howto & Style,"sprinkleofglitter|""sprinkle of glitter""|""Louis...",A big chat about everything going on in my pre...,238843,10028,248,https://i.ytimg.com/vi/OWvCCu2t6Ts/default.jpg,Great Britain
4,9GqL1DSQDL0,makeupbymichaelfinch,GET READY WITH ME + LILY! (we're so immature bye),26,Howto & Style,"Lily brown|""michael finch""|""makeupbymichaelfin...",Hey babes! Happy Friday! It's the WEEKENDDD He...,332967,22303,700,https://i.ytimg.com/vi/9GqL1DSQDL0/default.jpg,Great Britain


## **Combine Data US and GB**

In [14]:
data_youtube = pd.concat([df_us, df_gb])
sort_columns = ['video_id', 'channel_title', 'title', 'category_id', 'category_title', 'tags', 'description', 'views', 'likes', 'dislikes', 'thumbnail_link', 'country']
data_youtube = data_youtube[sort_columns]
data_youtube

,video_id,channel_title,title,category_id,category_title,tags,description,views,likes,dislikes,thumbnail_link,country
0,2kyS6SvSYSE,CaseyNeistat,WE WANT TO TALK ABOUT OUR MARRIAGE,22,People & Blogs,SHANtell martin,SHANTELL'S CHANNEL - https://www.youtube.com/s...,748374,57527,2966,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,USA
1,0mlNzVSJrT0,Nobrand,Me-O Cats Commercial,22,People & Blogs,"cute|""cats""|""thai""|""eggs""",Kittens come out of the eggs in a Thai commerc...,98966,2486,184,https://i.ytimg.com/vi/0mlNzVSJrT0/default.jpg,USA
2,STI2fI7sKMo,Shawn Johnson East,"AFFAIRS, EX BOYFRIENDS, $18MILLION NET WORTH -...",22,People & Blogs,"shawn johnson|""andrew east""|""shawn east""|""shaw...",Subscribe for weekly videos ▶ http://bit.ly/sj...,321053,4451,1772,https://i.ytimg.com/vi/STI2fI7sKMo/default.jpg,USA
3,KODzih-pYlU,Grace Helbig,BLIND(folded) CAKE DECORATING CONTEST (with Mo...,22,People & Blogs,"itsgrace|""funny""|""comedy""|""vlog""|""grace""|""helb...",Molly is an god damn amazing human and she cha...,197062,7250,217,https://i.ytimg.com/vi/KODzih-pYlU/default.jpg,USA
4,8mhTWqWlQzU,Safiya Nygaard,Wearing Online Dollar Store Makeup For A Week,22,People & Blogs,wearing online dollar store makeup for a week|...,I found this online dollar store called ShopMi...,2744430,115426,1110,https://i.ytimg.com/vi/8mhTWqWlQzU/default.jpg,USA
...,...,...,...,...,...,...,...,...,...,...,...,...
38821,JtIY1Naxg0M,Bleacher Report,Game of Zones - S5E7: NBA 1K,43,Shows,"bleacher report|""br""|""nba""|""philadelphia 76ers...",NBA rookies are roasting each other over a gam...,1664981,44245,357,https://i.ytimg.com/vi/JtIY1Naxg0M/default.jpg,Great Britain
38822,JtIY1Naxg0M,Bleacher Report,Game of Zones - S5E7: NBA 1K,43,Shows,"bleacher report|""br""|""nba""|""philadelphia 76ers...",NBA rookies are roasting each other over a gam...,1673959,44316,357,https://i.ytimg.com/vi/JtIY1Naxg0M/default.jpg,Great Britain
38823,JtIY1Naxg0M,Bleacher Report,Game of Zones - S5E7: NBA 1K,43,Shows,"bleacher report|""br""|""nba""|""philadelphia 76ers...",NBA rookies are roasting each other over a gam...,1686873,44434,360,https://i.ytimg.com/vi/JtIY1Naxg0M/default.jpg,Great Britain
38824,JtIY1Naxg0M,Bleacher Report,Game of Zones - S5E7: NBA 1K,43,Shows,"bleacher report|""br""|""nba""|""philadelphia 76ers...",NBA rookies are roasting each other over a gam...,1699351,44506,361,https://i.ytimg.com/vi/JtIY1Naxg0M/default.jpg,Great Britain


## **Text Processing**

**Check Missing Value**

In [15]:
data_youtube.columns[data_youtube.isna().any()].tolist()

['description']

In [16]:
data_youtube.isnull().sum()

video_id             0
channel_title        0
title                0
category_id          0
category_title       0
tags                 0
description       1130
views                0
likes                0
dislikes             0
thumbnail_link       0
country              0
dtype: int64

**Fill the NAN Column using fillna**

In [0]:
data_youtube['description'].fillna("No Description", inplace=True)

**Check the missing value again**

In [18]:
data_youtube.columns[data_youtube.isna().any()].tolist()

[]

In [19]:
data_youtube.isnull().sum()

video_id          0
channel_title     0
title             0
category_id       0
category_title    0
tags              0
description       0
views             0
likes             0
dislikes          0
thumbnail_link    0
country           0
dtype: int64

In [20]:
data_youtube.shape

(79775, 12)

In [21]:
data_youtube.tail(5)

,video_id,channel_title,title,category_id,category_title,tags,description,views,likes,dislikes,thumbnail_link,country
38821,JtIY1Naxg0M,Bleacher Report,Game of Zones - S5E7: NBA 1K,43,Shows,"bleacher report|""br""|""nba""|""philadelphia 76ers...",NBA rookies are roasting each other over a gam...,1664981,44245,357,https://i.ytimg.com/vi/JtIY1Naxg0M/default.jpg,Great Britain
38822,JtIY1Naxg0M,Bleacher Report,Game of Zones - S5E7: NBA 1K,43,Shows,"bleacher report|""br""|""nba""|""philadelphia 76ers...",NBA rookies are roasting each other over a gam...,1673959,44316,357,https://i.ytimg.com/vi/JtIY1Naxg0M/default.jpg,Great Britain
38823,JtIY1Naxg0M,Bleacher Report,Game of Zones - S5E7: NBA 1K,43,Shows,"bleacher report|""br""|""nba""|""philadelphia 76ers...",NBA rookies are roasting each other over a gam...,1686873,44434,360,https://i.ytimg.com/vi/JtIY1Naxg0M/default.jpg,Great Britain
38824,JtIY1Naxg0M,Bleacher Report,Game of Zones - S5E7: NBA 1K,43,Shows,"bleacher report|""br""|""nba""|""philadelphia 76ers...",NBA rookies are roasting each other over a gam...,1699351,44506,361,https://i.ytimg.com/vi/JtIY1Naxg0M/default.jpg,Great Britain
38825,JtIY1Naxg0M,Bleacher Report,Game of Zones - S5E7: NBA 1K,43,Shows,"bleacher report|""br""|""nba""|""philadelphia 76ers...",NBA rookies are roasting each other over a gam...,1709880,44588,360,https://i.ytimg.com/vi/JtIY1Naxg0M/default.jpg,Great Britain


**Data Cleaning**

In [22]:
data_youtube = data_youtube.drop_duplicates(subset='video_id', keep='first')
data_youtube.shape

(8602, 12)

In [23]:
data_youtube.head()

,video_id,channel_title,title,category_id,category_title,tags,description,views,likes,dislikes,thumbnail_link,country
0,2kyS6SvSYSE,CaseyNeistat,WE WANT TO TALK ABOUT OUR MARRIAGE,22,People & Blogs,SHANtell martin,SHANTELL'S CHANNEL - https://www.youtube.com/s...,748374,57527,2966,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,USA
1,0mlNzVSJrT0,Nobrand,Me-O Cats Commercial,22,People & Blogs,"cute|""cats""|""thai""|""eggs""",Kittens come out of the eggs in a Thai commerc...,98966,2486,184,https://i.ytimg.com/vi/0mlNzVSJrT0/default.jpg,USA
2,STI2fI7sKMo,Shawn Johnson East,"AFFAIRS, EX BOYFRIENDS, $18MILLION NET WORTH -...",22,People & Blogs,"shawn johnson|""andrew east""|""shawn east""|""shaw...",Subscribe for weekly videos ▶ http://bit.ly/sj...,321053,4451,1772,https://i.ytimg.com/vi/STI2fI7sKMo/default.jpg,USA
3,KODzih-pYlU,Grace Helbig,BLIND(folded) CAKE DECORATING CONTEST (with Mo...,22,People & Blogs,"itsgrace|""funny""|""comedy""|""vlog""|""grace""|""helb...",Molly is an god damn amazing human and she cha...,197062,7250,217,https://i.ytimg.com/vi/KODzih-pYlU/default.jpg,USA
4,8mhTWqWlQzU,Safiya Nygaard,Wearing Online Dollar Store Makeup For A Week,22,People & Blogs,wearing online dollar store makeup for a week|...,I found this online dollar store called ShopMi...,2744430,115426,1110,https://i.ytimg.com/vi/8mhTWqWlQzU/default.jpg,USA


In [0]:
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None

In [0]:
data_youtube['title'] = data_youtube['title'].str.lower()
data_youtube['title'] = data_youtube['title'].str.replace('[^\w\s]','')
data_youtube['description'] = data_youtube['description'].apply(lambda x: re.split('https:\/\/.*|http:\/\/.*', str(x))[0])
data_youtube['description'] = data_youtube['description'].str.lower()
data_youtube['description'] = data_youtube['description'].str.replace('[^\w\s]','')
data_youtube['tags'] = data_youtube['tags'].str.replace('\"*(\|")',',')
data_youtube['tags'] = data_youtube['tags'].str.replace('\"','')

In [26]:
data_youtube.head(3)

,video_id,channel_title,title,category_id,category_title,tags,description,views,likes,dislikes,thumbnail_link,country
0,2kyS6SvSYSE,CaseyNeistat,we want to talk about our marriage,22,People & Blogs,SHANtell martin,shantells channel,748374,57527,2966,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,USA
1,0mlNzVSJrT0,Nobrand,meo cats commercial,22,People & Blogs,"cute,cats,thai,eggs",kittens come out of the eggs in a thai commerc...,98966,2486,184,https://i.ytimg.com/vi/0mlNzVSJrT0/default.jpg,USA
2,STI2fI7sKMo,Shawn Johnson East,affairs ex boyfriends 18million net worth goo...,22,People & Blogs,"shawn johnson,andrew east,shawn east,shawn and...",subscribe for weekly videos,321053,4451,1772,https://i.ytimg.com/vi/STI2fI7sKMo/default.jpg,USA


**TFIDF**

In [0]:
selected_features = ['channel_title', 'title', 'category_title', 'tags', 'description', 'country']
data_youtube = data_youtube[selected_features]

In [28]:
data_youtube.head()

,channel_title,title,category_title,tags,description,country
0,CaseyNeistat,we want to talk about our marriage,People & Blogs,SHANtell martin,shantells channel,USA
1,Nobrand,meo cats commercial,People & Blogs,"cute,cats,thai,eggs",kittens come out of the eggs in a thai commerc...,USA
2,Shawn Johnson East,affairs ex boyfriends 18million net worth goo...,People & Blogs,"shawn johnson,andrew east,shawn east,shawn and...",subscribe for weekly videos,USA
3,Grace Helbig,blindfolded cake decorating contest with molly...,People & Blogs,"itsgrace,funny,comedy,vlog,grace,helbig,graceh...",molly is an god damn amazing human and she cha...,USA
4,Safiya Nygaard,wearing online dollar store makeup for a week,People & Blogs,"wearing online dollar store makeup for a week,...",i found this online dollar store called shopmi...,USA


Test 1 (Title and Description)

In [0]:
data_youtube.reset_index(inplace=True)

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data_youtube['description'])
tfidf_matrix.shape

(8602, 38552)

In [0]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [32]:
indices = pd.Series(data_youtube.index, index=data_youtube['title']).drop_duplicates()
indices

title
we want to talk about our marriage                                                                   0
meo cats commercial                                                                                  1
affairs ex boyfriends 18million net worth  google us  shawn and andrew                               2
blindfolded cake decorating contest with molly burke  grace helbig                                   3
wearing online dollar store makeup for a week                                                        4
                                                                                                  ... 
nikuman  magic japanese steam buns                                                                8597
wild about argyll                                                                                 8598
shane macgowan  nick cave  summer in siam  the wild mountain thyme  shanes 60th birthday party    8599
tourism australia dundee super bowl ad 2018 w chris hemsworth and d

In [0]:
def recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    data_youtube_indices = [i[0] for i in sim_scores]
    return data_youtube['title'].iloc[data_youtube_indices]

In [34]:
recommendations("wearing online dollar store makeup for a week")

381      i bought a full face of makeup from facebook ads
7966                i tried a full face of 7eleven makeup
6455    valentines day makeup tutorial glowing date ma...
58      lets talk money  everyday smokey eye tutorial ...
6093               how my makeup looks under a microscope
Name: title, dtype: object

In [36]:
data_youtube.iloc[381]

index                                                          2028
channel_title                                        Safiya Nygaard
title              i bought a full face of makeup from facebook ads
category_title                                       People & Blogs
tags              i bought a full face of makeup from facebook a...
description       so after buying the first 5 things from differ...
country                                                         USA
Name: 381, dtype: object

Test2 (Title and Category Title)

In [37]:
tfidf1 = TfidfVectorizer(stop_words='english')
tfidf_matrix1 = tfidf1.fit_transform(data_youtube['category_title'])
tfidf_matrix1.shape

(8602, 25)

In [0]:
cosine_sim1 = linear_kernel(tfidf_matrix1, tfidf_matrix1)

In [40]:
indices1 = pd.Series(data_youtube.index, index=data_youtube['title']).drop_duplicates()
indices1

title
we want to talk about our marriage                                                                   0
meo cats commercial                                                                                  1
affairs ex boyfriends 18million net worth  google us  shawn and andrew                               2
blindfolded cake decorating contest with molly burke  grace helbig                                   3
wearing online dollar store makeup for a week                                                        4
                                                                                                  ... 
nikuman  magic japanese steam buns                                                                8597
wild about argyll                                                                                 8598
shane macgowan  nick cave  summer in siam  the wild mountain thyme  shanes 60th birthday party    8599
tourism australia dundee super bowl ad 2018 w chris hemsworth and d

In [0]:
def recommendations1(title, cosine_sim=cosine_sim1):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    data_youtube_indices = [i[0] for i in sim_scores]
    return data_youtube['title'].iloc[data_youtube_indices]

In [42]:
recommendations1("affairs ex boyfriends 18million net worth  google us  shawn and andrew")

1                                   meo cats commercial
2     affairs ex boyfriends 18million net worth  goo...
3     blindfolded cake decorating contest with molly...
4         wearing online dollar store makeup for a week
5            mark ruffalo reads hilarious thirst tweets
6                idiots guide to japanese squat toilets
7     russian bots and trolls  november 8 2017 act 2...
8           the ultimate red lip look  nicole guerriero
9              could anastasia become a disney princess
10                           mdna skin express yourself
Name: title, dtype: object

In [44]:
data_youtube.iloc[5]

index                                                             5
channel_title                                        BuzzFeed Celeb
title                    mark ruffalo reads hilarious thirst tweets
category_title                                       People & Blogs
tags              BuzzFeed,buzzfeed celeb,Mark Ruffalo,Thor,Ragn...
description                           im justim flatterednncredits 
country                                                         USA
Name: 5, dtype: object